In [1]:
import cv2
import os
import numpy as np 
from IPython.display import Image  # for displaying images
from PIL import Image, ImageDraw, ImageFilter 
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import easyocr
import matplotlib.pyplot as plt
import imutils

In [ ]:
#restructerd images from datasets 